In [10]:
# Import libraries
from datetime import date
# Third party library for elexon - https://github.com/SheffieldSolar/Elexon-BMRS-API
from bmrs_api import BMRSDownloader
import pandas as pd

In [11]:
# Initializing api client
bmrs = BMRSDownloader(api_key="fq5wqyjmzynb1uu")

In [12]:
# Defining start and end dates
start = pd.Timestamp('202201010000', tz='UTC')
end = pd.Timestamp('202201020000', tz='UTC')

In [13]:
# Downloading data for the specified timeperiod
df = bmrs.download("B1620",start=start.date(), end=end.date())
# Retaining only data for period in consideration
df = df[df['datetime'] < str(end)]

In [14]:
# Initialzing empty list to append generation values
generation_list = []
for i, df_ in df.groupby(['datetime']):
    d = dict(zip(df_['PowerSystemResourceType'], df_['Quantity']))
    d['datetime'] = i
    generation_list.append(d)
# Converting generation list to pandas dataframe
df_generation = pd.DataFrame(generation_list)

In [15]:
# Creating a dictionary to help aggregate data
group_dict = {i:'mean' for i in df_generation.columns if i != 'datetime'}
# Creating a dictionary to update column names to have MW units
column_dict = {i:i+' (MW)' for i in df_generation.columns if i != 'datetime'}

In [16]:
# Aggregating data by 1 hour instead of the standard 30 mins
df_generation = df_generation.groupby(pd.Grouper(freq='1H', key='datetime')).agg(group_dict)
df_generation = df_generation.reset_index()

In [17]:
df_generation.rename(column_dict, axis=1, inplace=True)

In [18]:
# Writing output to csv
df_generation.to_csv('generation_by_production_type.csv', index=False)

In [19]:
df_generation

,datetime,Biomass (MW),Hydro Pumped Storage (MW),Hydro Run-of-river and poundage (MW),Fossil Hard coal (MW),Fossil Gas (MW),Fossil Oil (MW),Nuclear (MW),Other (MW),Wind Onshore (MW),Wind Offshore (MW),Solar (MW)
0,2022-01-01 00:00:00+00:00,1546.0,0.0,288.0,0.0,2848.0,0.0,6803.0,68.0,4653.6280,4712.6360,0.0
1,2022-01-01 01:00:00+00:00,1549.5,0.0,271.5,0.0,2877.0,0.0,6804.5,148.0,4972.5520,4914.7775,1.0
2,2022-01-01 02:00:00+00:00,1564.0,0.0,262.0,0.0,3050.0,0.0,6804.0,147.5,5738.2640,5081.1265,0.0
3,2022-01-01 03:00:00+00:00,1548.5,0.0,223.5,0.0,3242.5,0.0,6796.0,63.5,5859.1605,5296.6160,0.0
4,2022-01-01 04:00:00+00:00,1534.0,0.0,219.0,0.0,2893.5,0.0,6799.0,68.5,5774.7560,5758.7145,0.0
5,2022-01-01 05:00:00+00:00,1541.0,0.0,230.5,0.0,2744.5,0.0,6796.5,55.5,5419.5085,6014.9790,0.0
6,2022-01-01 06:00:00+00:00,1594.5,0.0,220.0,0.0,2671.0,0.0,6797.5,60.0,5039.9415,6375.8335,0.0
7,2022-01-01 07:00:00+00:00,1606.5,0.0,234.0,0.0,2537.0,0.0,6804.0,55.5,5242.2775,6534.1135,0.5
8,2022-01-01 08:00:00+00:00,1487.5,0.0,277.5,0.0,2582.5,0.0,6805.5,66.0,5152.7970,6705.3340,0.5
9,2022-01-01 09:00:00+00:00,1330.5,0.0,285.0,0.0,2724.0,0.0,6811.0,58.0,5635.3595,6719.4280,198.5
